In [1]:
from google.colab import files

# Upload the kaggle.json file that you downloaded
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [2]:
!pip install kaggle


In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [4]:
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d gauravduttakiit/food-image-classification


100% 1.82G/1.82G [01:36<00:00, 22.9MB/s]
100% 1.82G/1.82G [01:36<00:00, 20.4MB/s]


In [6]:
!unzip food-image-classification.zip


Streaming output truncated to the last 5000 lines.
  inflating: train/fried_rice/images_6220e787ec7f4fb39165a5ab6fa6c47a.jpg  
  inflating: train/ice_cream/1004744_8e16c351ccec4ef2bf4558e1071ece97.jpg  
  inflating: train/ice_cream/1012277_c99eb3a834844af9ad1a0ede6af393fb.jpg  
  inflating: train/ice_cream/101858_6873547bd58340fcbda1a419e3c5979c.jpg  
  inflating: train/ice_cream/1031536_61f0bcd0c68947aebaa212b4a9e89506.jpg  
  inflating: train/ice_cream/1044117_ce5c6673e4bb4d89b92bb4b9c7310ccd.jpg  
  inflating: train/ice_cream/1050427_6be9482b73214e6a82b9dd54c17839a6.jpg  
  inflating: train/ice_cream/1053680_18e9a4cd145441a48e02d36ddcd619eb.jpg  
  inflating: train/ice_cream/1069788_177942a731f745cebad04c17427d8dd5.jpg  
  inflating: train/ice_cream/1075106_44ee85e0fe88491086fa9a68566f291a.jpg  
  inflating: train/ice_cream/1078154_f21722646c174489bcce731485c01892.jpg  
  inflating: train/ice_cream/1100243_2c32233b6ac64f9db999b3b276bea174.jpg  
  inflating: train/ice_cream/1101478_b

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split

In [8]:
Id=[]
import os
for dirname, _, filenames in os.walk(r'/content/test'):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))
Id[:5]

train=pd.DataFrame()
train=train.assign(filename=Id)
train.head()

train['label']=train['filename']
train['label']=train['label'].str.replace(r'/content/test','')
train.head()



,filename,label
0,/content/test/dhokla/259_0de3bd37f3784cb9a1503...,/dhokla/259_0de3bd37f3784cb9a150355994c23fbb.jpg
1,/content/test/dhokla/143_ed649494abb24696ab52a...,/dhokla/143_ed649494abb24696ab52acddbab7f0f2.jpg
2,/content/test/dhokla/218_33e9b91f304c4b49a46a9...,/dhokla/218_33e9b91f304c4b49a46a9b6c734fb837.jpg
3,/content/test/dhokla/023_6be6c29d33d84db09e082...,/dhokla/023_6be6c29d33d84db09e082e3650d0a236.jpg
4,/content/test/dhokla/208_93f13f7e0dba4b3eac8be...,/dhokla/208_93f13f7e0dba4b3eac8beda68895ffde.jpg


In [9]:
train['label'] = train['label'].str.extract(r'/([^/]+)/')
train.head()


,filename,label
0,/content/test/dhokla/259_0de3bd37f3784cb9a1503...,dhokla
1,/content/test/dhokla/143_ed649494abb24696ab52a...,dhokla
2,/content/test/dhokla/218_33e9b91f304c4b49a46a9...,dhokla
3,/content/test/dhokla/023_6be6c29d33d84db09e082...,dhokla
4,/content/test/dhokla/208_93f13f7e0dba4b3eac8be...,dhokla


In [10]:
# Step 1: Load the dataset
data = train

# Step 2: Data Preprocessing
# ImageDataGenerator for preprocessing and data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    validation_split=0.2  # Splitting data into training and validation sets
)

# Splitting data into training and validation sets
train_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col="filename",
    y_col="label",
    target_size=(512,512),  # Resizing images to 224x224
    batch_size=10,
    subset="training",
    class_mode="categorical"
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col="filename",
    y_col="label",
    target_size=(512, 512),
    batch_size=5,
    subset="validation",
    class_mode="categorical"
)

# Step 3: Define the ResNet Model
resnet_model = Sequential()
resnet_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))  # Pre-trained ResNet50 model
resnet_model.add(Dense(len(train_generator.class_indices), activation='softmax'))  # Output layer

# Step 4: Train the Model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(train_generator, validation_data=valid_generator, epochs=35)


Found 3820 validated image filenames belonging to 34 classes.
Found 955 validated image filenames belonging to 34 classes.
94765736/94765736 [==============================] - 5s 0us/step
Epoch 1/35
382/382 [==============================] - 270s 594ms/step - loss: 2.9584 - accuracy: 0.1257 - val_loss: 4.4000 - val_accuracy: 0.0000e+00
Epoch 2/35
382/382 [==============================] - 223s 583ms/step - loss: 2.3414 - accuracy: 0.2421 - val_loss: 25.8882 - val_accuracy: 0.0000e+00
Epoch 3/35
382/382 [==============================] - 223s 583ms/step - loss: 2.0579 - accuracy: 0.3123 - val_loss: 7.3624 - val_accuracy: 0.0000e+00
Epoch 4/35
382/382 [==============================] - 227s 594ms/step - loss: 1.9093 - accuracy: 0.3623 - val_loss: 7.9334 - val_accuracy: 0.0000e+00
Epoch 5/35
382/382 [==============================] - 223s 583ms/step - loss: 1.7699 - accuracy: 0.4160 - val_loss: 14.3747 - val_accuracy: 0.0052
Epoch 6/35
382/382 [==============================] - 222s 582ms